### Here we will implement LSTM model.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds

In [3]:
imdb,info=tfds.load('imdb_reviews/subwords8k',with_info=True,as_supervised=True)
features=info.features['text']

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteVPPPXE/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteVPPPXE/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteVPPPXE/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0. Subsequent calls will reuse this data.


In [ ]:
sub_token=features.encoder
subwords=sub_token.subwords

In [5]:
subwords[:10]

['the_', ', ', '. ', 'a_', 'and_', 'of_', 'to_', 's_', 'is_', 'br']

In [ ]:
train_data,test_data=imdb['train'], imdb['test']

train_batch=(train_data.shuffle(1000).padded_batch(64,tf.compat.v1.data.get_output_shapes(train_data)))
test_batch=(test_data.padded_batch(64,tf.compat.v1.data.get_output_shapes(test_data)))

In [59]:
sub_token.decode(i[2])

"That's how I'd sum it up! There were other who said, Naked Gun meets James Bond. I think mine is more accurate, but I'm not neutral on that point ... ;o) <br /><br />This movie is kind of a blast from the past. It reminds you of the old movies (from the '50s & '60s) and that's not only because it plays in that time period (1955 to be exact, with the Flashback exception), but it's the mood the movie is made in and it shows on screen! The main actor here (who is as I've been told very popular in his homeland France), is perfect in his role here. He's the spy with the number 117 attached to him. There is one character who sums him up in the movie, by asking a question, about if he's good or not ... you'll see when you watch the movie! <br /><br />Very funny indeed this is and if you like your spies mixed with comedy with old school flair, but not taking itself or anything else for that matter seriously, than you need to watch this! :o)"

In [ ]:
class MyModel():
    def __init__(self,vocab_size,max_length,embed_dim,tensorflow,keras,layers):
        self.vocab_size=vocab_size
        self.max_length=max_length
        self.embed_dim=embed_dim
        self.tensorflow=tensorflow
        self.keras=keras
        self.layers=layers
    
    def single_LSTM(self):
        model=self.keras.Sequential()
        model.add(self.layers.Embedding(input_dim=vocab_size,output_dim=embed_dim))
        model.add(self.layers.Bidirectional(self.layers.LSTM(embed_dim)))
        model.add(self.layers.Dense(units=6,activation='relu'))
        model.add(self.layers.Dense(units=1,activation='sigmoid'))
        model.compile(optimizer='adam',
                     loss=self.keras.losses.BinaryCrossentropy(from_logits=True),
                     metrics=['accuracy'])
        return model
    ### WE cannot use classic for train_batch and test batch
    ### Because for Flatten we have to specify input_lenght which varies for batch to batch
    ### If we want to use classic with Flatten we have to break batch to category and labels
    ### and then we have to try 
    def classic(self):
        model=self.keras.Sequential()
        model.add(self.layers.Embedding(input_dim=vocab_size,output_dim=embed_dim))
        model.add(self.layers.GlobalAveragePooling1D())
        model.add(self.layers.Dense(units=6,activation='relu'))
        model.add(self.layers.Dense(units=1,activation='sigmoid'))
        model.compile(optimizer='adam',
                     loss=self.keras.losses.BinaryCrossentropy(from_logits=True),
                     metrics=['accuracy'])
        return model
    
    def classic_Globalaverage(self):
        model=self.keras.Sequential()
        model.add(self.layers.Embedding(input_dim=vocab_size,output_dim=embed_dim))
        model.add(self.layers.GlobalAveragePooling1D())
        model.add(self.layers.Dense(units=6,activation='relu'))
        model.add(self.layers.Dense(units=1,activation='sigmoid'))
        model.compile(optimizer='adam',
                     loss=self.keras.losses.BinaryCrossentropy(from_logits=True),
                     metrics=['accuracy'])
        return model 
    
    def classic_Convo(self):
        model=self.keras.Sequential()
        model.add(self.layers.Embedding(input_dim=vocab_size,output_dim=embed_dim))
        model.add(self.layers.Conv1D(128,5,activation='relu'))
        model.add(self.layers.GlobalMaxPool1D())
        model.add(self.layers.Dense(units=6,activation='relu'))
        model.add(self.layers.Dense(units=1,activation='sigmoid'))
        model.compile(optimizer='adam',
                     loss=self.keras.losses.BinaryCrossentropy(from_logits=True),
                     metrics=['accuracy'])
        return model 
    

In [ ]:
vocab_size=sub_token.vocab_size
embed_dim=16
max_len=64

In [ ]:
models=MyModel(vocab_size,embed_dim,max_len,tf,tf.keras,tf.keras.layers)


In [17]:

lstm_model=models.single_LSTM()

lstm_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 16)          130960    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 32)                4224      
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 198       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 7         
Total params: 135,389
Trainable params: 135,389
Non-trainable params: 0
_________________________________________________________________


In [10]:
lstm_model.fit(train_batch,epochs=10,validation_data=test_batch)

Epoch 1/10
391/391 [==============================] - 53s 135ms/step - loss: 0.6880 - accuracy: 0.5238 - val_loss: 0.6929 - val_accuracy: 0.5002
Epoch 2/10
391/391 [==============================] - 52s 133ms/step - loss: 0.6336 - accuracy: 0.6942 - val_loss: 0.6201 - val_accuracy: 0.7316
Epoch 3/10
391/391 [==============================] - 53s 135ms/step - loss: 0.6516 - accuracy: 0.6272 - val_loss: 0.6929 - val_accuracy: 0.5008
Epoch 4/10
391/391 [==============================] - 52s 132ms/step - loss: 0.6929 - accuracy: 0.5006 - val_loss: 0.6929 - val_accuracy: 0.5008
Epoch 5/10
391/391 [==============================] - 52s 134ms/step - loss: 0.6929 - accuracy: 0.5007 - val_loss: 0.6929 - val_accuracy: 0.5008
Epoch 6/10
391/391 [==============================] - 52s 133ms/step - loss: 0.6928 - accuracy: 0.5010 - val_loss: 0.6928 - val_accuracy: 0.5009
Epoch 7/10
391/391 [==============================] - 52s 133ms/step - loss: 0.6736 - accuracy: 0.5616 - val_loss: 0.6424 - val_ac

In [ ]:
### Lets try classic

In [93]:
classic=models.classic()
classic.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, None, 16)          130960    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense_12 (Dense)             (None, 6)                 102       
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 7         
Total params: 131,069
Trainable params: 131,069
Non-trainable params: 0
_________________________________________________________________


In [94]:
classic.fit(train_batch,epochs=10,validation_data=(test_batch))

Epoch 1/10
391/391 [==============================] - 9s 22ms/step - loss: 0.7034 - accuracy: 0.5000 - val_loss: 0.6935 - val_accuracy: 0.5000
Epoch 2/10
391/391 [==============================] - 8s 21ms/step - loss: 0.6925 - accuracy: 0.5000 - val_loss: 0.6922 - val_accuracy: 0.5000
Epoch 3/10
391/391 [==============================] - 8s 21ms/step - loss: 0.6914 - accuracy: 0.5000 - val_loss: 0.6911 - val_accuracy: 0.5000
Epoch 4/10
391/391 [==============================] - 8s 21ms/step - loss: 0.6882 - accuracy: 0.5079 - val_loss: 0.6849 - val_accuracy: 0.5176
Epoch 5/10
391/391 [==============================] - 8s 21ms/step - loss: 0.6660 - accuracy: 0.5847 - val_loss: 0.6488 - val_accuracy: 0.6638
Epoch 6/10
391/391 [==============================] - 8s 21ms/step - loss: 0.6283 - accuracy: 0.7461 - val_loss: 0.6203 - val_accuracy: 0.7862
Epoch 7/10
391/391 [==============================] - 8s 21ms/step - loss: 0.6032 - accuracy: 0.8160 - val_loss: 0.6023 - val_accuracy: 0.8160